In [1]:
!pip install confluent-kafka

     |████████████████████████████████| 8.1 MB 13.0 MB/s eta 0:00:01


In [2]:
!pip install confluent-kafka[avro]

     |████████████████████████████████| 2.4 MB 13.2 MB/s eta 0:00:01
  Created wheel for avro-python3: filename=avro_python3-1.10.0-py3-none-any.whl size=43735 sha256=61bfa4916b1ebe4aff0f9de43dcd9dc7cb5e21ceebc8d6202f341ed221b3b1f9
  Stored in directory: /home/jovyan/.cache/pip/wheels/52/db/5d/a6a6e9cd4c5d626432ce8258a0141de4f34cd7e68d79e9c4f4
Successfully built avro-python3


In [3]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [4]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [5]:
conf = {'bootstrap.servers': bootstrap_server}

In [6]:
a = AdminClient(conf)

In [7]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    ##if not (str(t)).startswith("_"):
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 4 topics:
  "_confluent-ksql-confluent_rmoff_01_command_topic" with 1 partition(s)
  "__transaction_state" with 50 partition(s)
  "_schemas" with 1 partition(s)
  "confluent_rmoff_01ksql_processing_log" with 1 partition(s)


In [8]:
#fs = a.create_topics([NewTopic("test1p", num_partitions=1, replication_factor=1)])

#for topic, f in fs.items():
#    try:
#        f.result()  # The result itself is None
#        print("Topic {} created".format(topic))
#    except Exception as e:
#        print("Failed to create topic {}: {}".format(topic, e))

## Define the value schema

In [9]:
value_schema_str = """
{
  "namespace": "hoven",
  "type": "record",
  "name": "iotvalue",
  "fields": [
    {
      "name": "component",
      "type": "string"
    },
    {
      "name": "sensor",
      "type": "string"
    },
    {
      "name": "temperature",
      "type": "float"
    },
    {
      "name": "humidity",
      "type": "float"
    },
    {
      "name": "ts",
      "type": "long"
    }
  ]
}
"""

In [10]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
import time

topic = "iot-oven"
schema_registry_url = "http://schema-registry:8081"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [11]:
schema_registry_conf = {'url': schema_registry_url}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [12]:
avro_serializer = AvroSerializer(value_schema_str, schema_registry_client)

producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': avro_serializer
}

producer = SerializingProducer(producer_conf)

In [ ]:
from random import gauss
i = 0;

while True:
    i += 1
    key = "S1"
    value = {"component": "iot-oven", "sensor": "S1","temperature": gauss(290, 5.0),"humidity": gauss(30, 5.0),"ts":int(time.time()*1000.0)}
    producer.produce(topic=topic, value=value, key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)


{'component': 'iot-oven', 'sensor': 'S1', 'temperature': 290.70097793787977, 'humidity': 32.64539192193882, 'ts': 1605541156955}
{'component': 'iot-oven', 'sensor': 'S1', 'temperature': 289.14064039672047, 'humidity': 28.55293753871481, 'ts': 1605541168297}
Produced record to topic iot-oven partition [0] @ offset 0
{'component': 'iot-oven', 'sensor': 'S1', 'temperature': 283.23814508500084, 'humidity': 33.759427757090556, 'ts': 1605541178308}
Produced record to topic iot-oven partition [0] @ offset 1
{'component': 'iot-oven', 'sensor': 'S1', 'temperature': 287.1422056584823, 'humidity': 35.420375513639634, 'ts': 1605541188313}
Produced record to topic iot-oven partition [0] @ offset 2
{'component': 'iot-oven', 'sensor': 'S1', 'temperature': 286.68198157709406, 'humidity': 29.757238856180525, 'ts': 1605541198314}
Produced record to topic iot-oven partition [0] @ offset 3
{'component': 'iot-oven', 'sensor': 'S1', 'temperature': 289.1672206236058, 'humidity': 35.952458825870536, 'ts': 160